#### **Awards collection**
- Go through this list of movies (243 movies in total) and document the awards that a director in each movie has won

In [0]:
# make some preparation for packages
# package use
import requests
import urllib
import urllib.request
import time
import re
from bs4 import BeautifulSoup
from IPython.core.display import HTML

import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
import copy
import datetime 
from collections import Counter
import json

In [3]:
dataset = pd.read_csv("Test_Award_Director.csv")
dataset.head(5)

,year,bomtitle,imdblink
0,2005.0,Fantastic Four (2005),http://www.imdb.com/title/tt0120667/
1,2007.0,Awake,http://www.imdb.com/title/tt0211933/
2,2004.0,Therese: The Story of Saint Therese of Lisieux,http://www.imdb.com/title/tt0286218/
3,2007.0,Happily N'Ever After,http://www.imdb.com/title/tt0308353/
4,2004.0,Resident Evil: Apocalypse,http://www.imdb.com/title/tt0318627/


In [4]:
# first scrape directors' personal IMDb page
dataset['director'] = np.nan
dataset['director_link'] = np.nan

# filter script[type=application/ld+json] contents
def find_director_link(url_):
  name = np.nan
  link = np.nan
  source_code = requests.get(url_)
  plain_text = source_code.text
  soup = BeautifulSoup(plain_text, 'html.parser')
  js = json.loads(soup.select_one("script[type=application/ld+json]").text)
  try:
    if isinstance(js['director'], dict):
      # single 
      dictionary = js['director']
    else:
      # may multiple or nothing
      dictionary = js['director'][0]
    name = dictionary['name']
    link = "https://www.imdb.com" + dictionary['url']
  except:
    pass
  return name, link
# test
find_director_link(dataset['imdblink'][0])

('Tim Story', 'https://www.imdb.com/name/nm1103162/')

In [0]:
for i in range(len(dataset)):
  dataset['director'][i], dataset['director_link'][i] = find_director_link(dataset['imdblink'][i])
  # processing
  if i%30 == 0:
    print(i)

In [6]:
# check any blanks
print(len(dataset[dataset['director'].isna()]))
# they are movies without any directors content
dataset[dataset['director'].isna()]

5


,year,bomtitle,imdblink,director,director_link
58,NaN,Batman Begins (2012 re-release),http://www.imdb.com/title/tt0450392/,NaN,NaN
63,NaN,Star Wars: The Clone Wars,http://www.imdb.com/title/tt0458290/,NaN,NaN
89,NaN,Primeval,http://www.imdb.com/title/tt0808096/,NaN,NaN
201,NaN,The Fall,http://www.imdb.com/title/tt2294189/,NaN,NaN
209,NaN,2011 Oscar Shorts,http://www.imdb.com/title/tt2447024/,NaN,NaN


In [0]:
def awards_blurb_extraction(url_):
  result = np.nan
  # check if url nan
  try:
    source_code = requests.get(url_)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text, 'html.parser')
    filter_ = soup.find_all("span", {"class":"awards-blurb"})
    result = filter_[0].get_text().replace("\n","").replace(" ", "")
  except:
    pass
  return result

In [0]:
# Go into personal page for awards link
# and find number of awards and nomination
# initialization
dataset['awards_blurb'] = np.nan
for i in range(len(dataset)):
  dataset['awards_blurb'][i] = awards_blurb_extraction(dataset['director_link'][i])
  # process
  if i%50 == 0:
    print(i)

In [36]:
# check any blanks
print(len(dataset[dataset['awards_blurb'].isna()]))

143


In [0]:
# set up nominations and awards column
dataset['nominations'] = np.nan
dataset['awards'] = np.nan

# separate records from awards_blurb strings
# there are several possible formats
# Won/Nominated for ...
# Nominations/wins/win/Normation
def award_nomination_count(string_):
  awards = 0
  nomination = 0
  # first check either Won or Nominated
  string_ = str(string_)
  if "Nominated" in string_:
    temp = re.findall(r'\d+', string_)
    nomination = int(temp[0])
  if "Won" in string_:
    temp = re.findall(r'\d+', string_)
    awards = int(temp[0])

  # check either nomination/win/both
  nomination_find = False
  win_find = False
  if "nomination" in string_:
    temp = re.findall(r'\d+', string_)
    if "win" in string_: # both
      awards = int(temp[0])
      nomination = int(temp[1])
    else: # only nomination
      nomination = int(temp[0])
  elif "win" in string_:
    temp = re.findall(r'\d+', string_)
    awards = int(temp[0])
  return awards, nomination

In [0]:
for i in range(len(dataset)):
  str_content = dataset['awards_blurb'][i]
  dataset['awards'][i], dataset['nominations'][i] = award_nomination_count(str_content)

In [0]:
dataset.to_csv("Test_Award_Director_result.csv")